In [1]:
import pandas as pd
import numpy as np

from google.colab import files
uploaded = files.upload()

Saving 202501_clean2.xlsx to 202501_clean2.xlsx


In [10]:
file_clean = '202501_clean2.xlsx'
df = pd.read_excel(excel_path)

In [6]:
# 대상 컬럼 추출
target_columns = ['Conc(ug/m3)', 'Cr(ng/m3)', 'Co(ng/m3)', 'Ni(ng/m3)',
                  'As(ng/m3)', 'Cd(ng/m3)', 'Sb(ng/m3)', 'Pb(ng/m3)']

missing = [c for c in target_columns if c not in df.columns]
if missing:
    raise ValueError(f"[오류] 다음 컬럼이 데이터에 없음: {missing}\n엑셀의 컬럼명을 확인하세요.")

print("[확인] 대상 컬럼:", target_columns)
print("[확인] 원본 크기:", df.shape)
df[target_columns].head(6)

[확인] 대상 컬럼: ['Conc(ug/m3)', 'Cr(ng/m3)', 'Co(ng/m3)', 'Ni(ng/m3)', 'As(ng/m3)', 'Cd(ng/m3)', 'Sb(ng/m3)', 'Pb(ng/m3)']
[확인] 원본 크기: (461, 11)


,Conc(ug/m3),Cr(ng/m3),Co(ng/m3),Ni(ng/m3),As(ng/m3),Cd(ng/m3),Sb(ng/m3),Pb(ng/m3)
0,NaN,6.11,9.39,6.92,0.0,316.97,29.99,16.83
1,10.20,7.26,5.50,7.83,0.0,313.15,NaN,13.57
2,6.90,5.53,5.60,6.67,0.0,292.73,28.12,16.47
3,6.26,6.43,8.38,7.74,0.0,347.83,24.88,2.43
4,8.77,5.50,6.71,6.42,0.0,335.27,19.10,4.33
5,7.58,5.64,5.40,6.68,0.0,323.34,21.71,3.33


In [7]:
# 수치화
df[target_columns] = df[target_columns].apply(pd.to_numeric, errors='coerce')

# 음수값 NaN 처리
neg_counts = (df[target_columns] < 0).sum(numeric_only=True)
print("[정보] 음수값 개수(컬럼별):")
print(neg_counts)

df[target_columns] = df[target_columns].mask(df[target_columns] < 0, np.nan)

# 결측 개수 확인
nan_counts = df[target_columns].isna().sum()
print("\n[정보] NaN 개수(컬럼별, 음수 처리 후):")
print(nan_counts)

[정보] 음수값 개수(컬럼별):
Conc(ug/m3)    0
Cr(ng/m3)      0
Co(ng/m3)      0
Ni(ng/m3)      0
As(ng/m3)      0
Cd(ng/m3)      0
Sb(ng/m3)      0
Pb(ng/m3)      0
dtype: int64

[정보] NaN 개수(컬럼별, 음수 처리 후):
Conc(ug/m3)      1
Cr(ng/m3)      109
Co(ng/m3)       12
Ni(ng/m3)      135
As(ng/m3)       12
Cd(ng/m3)       12
Sb(ng/m3)      229
Pb(ng/m3)       66
dtype: int64


In [19]:
# NaN으로 처리된 원래 값들을 저장
nan_sources = {}

for col in target_columns:
    # 숫자로 변환했을 때 NaN이 된 위치
    mask_nan = pd.to_numeric(df_raw[col], errors='coerce').isna()
    # 그 위치에서 원래 값 확인
    unique_vals = df_raw.loc[mask_nan, col].dropna().unique()
    nan_sources[col] = unique_vals

# 결과 출력
for col, vals in nan_sources.items():
    print(f"[{col}] NaN 처리된 원래 값 → {vals if len(vals) > 0 else '빈칸(Blank cell)'}")

[Conc(ug/m3)] NaN 처리된 원래 값 → 빈칸(Blank cell)
[Cr(ng/m3)] NaN 처리된 원래 값 → 빈칸(Blank cell)
[Co(ng/m3)] NaN 처리된 원래 값 → 빈칸(Blank cell)
[Ni(ng/m3)] NaN 처리된 원래 값 → 빈칸(Blank cell)
[As(ng/m3)] NaN 처리된 원래 값 → 빈칸(Blank cell)
[Cd(ng/m3)] NaN 처리된 원래 값 → 빈칸(Blank cell)
[Sb(ng/m3)] NaN 처리된 원래 값 → 빈칸(Blank cell)
[Pb(ng/m3)] NaN 처리된 원래 값 → 빈칸(Blank cell)


In [21]:
# 데이터계열표 인덱스(행) 정의
index_rows = ['Mean', 'BestFit', 'AD', 'p-value']

# 결과 테이블 초기화: NaN으로 채움
series_table = pd.DataFrame(index=index_rows, columns=target_columns, dtype='object')

# 결측(NaN) 무시하고 평균값 계산
mean_values = df[target_columns].mean(numeric_only=True)

# 반올림
series_table.loc['Mean'] = mean_values.round(6).astype(object)

# 확률분포피팅 표 생성
# series_table.loc['BestFit']  # 빈 상태
# series_table.loc['AD']
# series_table.loc['p-value']

print("[완료] 데이터계열표 미리보기:")
series_table

[완료] 데이터계열표 미리보기:


,Conc(ug/m3),Cr(ng/m3),Co(ng/m3),Ni(ng/m3),As(ng/m3),Cd(ng/m3),Sb(ng/m3),Pb(ng/m3)
Mean,16.219087,2.969119,7.845969,2.974877,0.0,213.893252,17.279569,8.939797
BestFit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
p-value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
